#Preliminari


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [3]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 35 kB/s 
     |████████████████████████████████| 199 kB 54.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=52b34b495ae307d8f9caa7442c90323d243643c0c7f326539d00375f8dcd6bf5
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
findspark.init()

In [5]:
from pyspark import SparkContext, SparkConf

In [6]:
conf = SparkConf().setAppName('GEBD2').setMaster("local[*]")
sc = SparkContext(conf=conf)

#Importo dataset

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

DATASET BRIGHTKITE

In [9]:
path = "/content/drive/MyDrive/data2.csv"
df_bonus = pd.read_csv(path)

In [10]:
derived_df = df_bonus[['From', 'To']]

In [11]:
derived_df

,From,To
0,0,2
1,0,3
2,0,4
3,0,5
4,0,6
...,...,...
428150,58225,58226
428151,58225,58227
428152,58226,58220
428153,58226,58225


In [12]:
fromID = derived_df['From'].tolist()
toID = derived_df['To'].tolist()


In [13]:
l = len(fromID)
fromID1 = fromID
toID1 = toID
fromtoID = list()

In [14]:
for i in range(0,l):
     
     fromtoID.insert(i,((fromID1[i]),(toID1[i])))

fromtoID.append((0,1)) #nel dataset manca il primo arco, aggiungiamo manualmente


In [15]:
len(fromtoID)

428156

#Funzioni

In [16]:
import time
import numpy as np
from operator import add

def filter1(x):
  '''Dato un elemento nella forma <(i,j,a,b)>
     Restitusce <(i,j)> se a e b rispettano una determinata condizione.
     Funzione usata nel map1.
  '''
  if x[1]<x[2] or (x[1]==x[2] and x[0][0]<x[0][1]):
    return x

def to_list(a):
     return [a]
def append(a, b):
     a.append(b)
     return a
def extend(a, b):
     a.extend(b)
     return a
 
def trova_grado(x):
  '''Ammette in input l'etichetta di un nodo.
     Restituisce in output il relativo grado.
     Usata per facilitare la fase di map1.'''
  grado = int(gradinp[solonodi == x][0][1])
  return grado

def map2(x):
  '''La funzione map2(x) ammette in input un elemento del tipo
     <int(n),{n1,n2,..,nq} e restituisce in output
     <int(n),(ni,nj)> per ogni possibile coppia estratta dall'insieme
     {n1,n2,..,nq} tale che ni è dominato da nj.
  ''' 
  output = []
  nodo = x[0]
  nodo = int(nodo)
  alto_vicinato_x = x[1]
  len_alto_vicinato_x = len(alto_vicinato_x)
  
  k = 0
  j = -1

  for i in alto_vicinato_x[0:len(alto_vicinato_x)-1]:
    grado1 = trova_grado(i)
    nodo1 = i
    soglia =  len(alto_vicinato_x)-1
    j += 1
    k = 1 + j
 
    while k <= soglia:
      nodo2 = alto_vicinato_x[k]
      nodo2 = int(nodo2)
      grado2 = trova_grado(nodo2)

      if grado1 < grado2:
        output.append((nodo,(nodo1,nodo2)))
        k += 1
      elif grado1 == grado2 and nodo1 < nodo2:
        output.append((nodo,(nodo1,nodo2)))
        k += 1
      elif grado1 > grado2:
        output.append((nodo,(nodo2,nodo1)))
        k += 1
      else:
        output.append((nodo,(nodo2,nodo1)))
        k += 1

  return output

def map3(x):
  '''La funzione map3 ammette in input una RDD con una struttura <i,{n1,..,np}>
     e restituisce in output una RDD del tipo <(i,n1)>, .., <(i,np)>
  '''
  output=[]
  for i in x[1]:
    output.append( (i,x[0]) )
  return output

 

#Implementazione FFFk su grafo

In [17]:
def calcola_triangoli(lista_archi, direzionato=False):
 
 '''La funzione calcola_trinagoli() ammette come input
    la lista degli archi del grafo di interesse.
    Se il grafo è direzionato
    [ovvero esiste arco (i,j), ma non è presente (j,i)]
    impostare direzionato = True 
    in quanto l'algoritmo lavora su grafi non direzionati,
    ciò renderà il grafo non direzionato'''


 if direzionato == True:
   for i in lista_archi:
     lista_archi.append((i[1],i[0]))
 
 start = time.time()

 
 ### ROUND1 ###

 k = 3
  
 dArchi0 = lista_archi
 dArchi0 = sc.parallelize(dArchi0)

 # Map1
 # Elaboro gradi nodi
 dGradi = dArchi0.map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y) 
 # Salvo i gradi appena elaborati in una lista
 gradi = dGradi.collect()
 

 # Salvo nodi e gradi in due liste separate
 gradinp = np.array(gradi)
 solonodi = gradinp[0:,0]
 
 def trova_grado(x):
  '''Ammette in input l'etichetta di un nodo.
     Restituisce in output il relativo grado.
     Usata per facilitare la fase di map1.'''
  grado = int(gradinp[solonodi == x][0][1])
  return grado

 #input map 1: passo da <(i,j)> a <(i,j),grado(i),grado(j)>
 dArchi1 = dArchi0.map(lambda x:(x,trova_grado(x[0]),trova_grado(x[1])))
 

 # output map 1: filtro la metà degli elementi di dArchi1 in modo che
 # ciascun elemento della nuova RDD sia del tipo <i,j> con i dominato da j 
 dArchi2 = dArchi1.filter(lambda x:filter1(x)).map(lambda x: (x[0][0],x[0][1]))

 # trovo alto vicinato
 dArchi2_1 = dArchi2.combineByKey(to_list,append,extend)

 # output Reduce1: seleziono nodi x aventi cardinalità di gamma+(x)>=k-1
 dArchi3 = dArchi2_1.filter(lambda x:len(x[1])>=k-1)



 ### ROUND 2 ###
 
 def map2(x):
  '''La funzione map2(x) ammette in input un elemento del tipo
     <int(n),{n1,n2,..,nq} e restituisce in output
     <int(n),(ni,nj)> per ogni possibile coppia estratta dall'insieme
     {n1,n2,..,nq} tale che ni è dominato da nj.
  ''' 
  output = []
  nodo = x[0]
  nodo = int(nodo)
  alto_vicinato_x = x[1]
  len_alto_vicinato_x = len(alto_vicinato_x)
  
  k = 0
  j = -1

  for i in alto_vicinato_x[0:len(alto_vicinato_x)-1]:
    grado1 = trova_grado(i)
    nodo1 = i
    soglia =  len(alto_vicinato_x)-1
    j += 1
    k = 1 + j
 
    while k <= soglia:
      nodo2 = alto_vicinato_x[k]
      nodo2 = int(nodo2)
      grado2 = trova_grado(nodo2)

      if grado1 < grado2:
        output.append((nodo,(nodo1,nodo2)))
        k += 1
      elif grado1 == grado2 and nodo1 < nodo2:
        output.append((nodo,(nodo1,nodo2)))
        k += 1
      elif grado1 > grado2:
        output.append((nodo,(nodo2,nodo1)))
        k += 1
      else:
        output.append((nodo,(nodo2,nodo1)))
        k += 1

  return output

 # Map2
 
 #map2.1
 #output map 2 per input del tipo 1 (input tipo <u,gamma+(u)>) 
 #inverto poi chiave valore ottenendo output
 #Si passa da una struttura del tipo <n,{n1,..,nq}> a 
 #<(n,n1),n>,.., <(n,nq),n> per ogni n della RDD
 dArchi4 = dArchi3.flatMap(lambda x: map2(x)).map(lambda x: (x[1],x[0]))

 #map2.2 (senza <(u,v),0>). Riprendiamo output map1 e aggiungiamo $
 dArchi5=dArchi2.map(lambda x: (x,'$'))

 #output map2.1 (archi confermati)
 #Solo una parte degli archi in chiave di dArchi4 sono effettivamente archi,
 #facciamo un join per eliminare i "non archi"
 dArchi6 = dArchi5.join(dArchi4).map(lambda x:(x[0],x[1][1]))

 # Reduce2: un reducebykey sulla ultima RDD calcolata
 dArchi7 = dArchi6.combineByKey(to_list, append, extend)



 ### ROUND 3 ###

 # Map3: si passa da una struttura <(i,j),{n1,..,np}> a una struttura
 # <(i,j),n1>, .., <(i,j),np> per ogni (i,j) appartenente a dArchi7
 
 dArchi8 = dArchi7.flatMap(lambda x: map3(x))

 # Reduce3: per ciascuna chiave di dArchi8 
 # si conta il numero di elementi presenti nella parte valore della rispettiva
 # chiave
 dArchi9 = dArchi8.map(lambda x: (x[0],len([x[0]])))

 dArchi10=dArchi9.reduceByKey(lambda x,y: x+y)

 # Calcolo numero clique come somma di queste "taglie"
 numeroTriangoli = dArchi10.map(lambda x: x[1]).reduce(add)
 end = time.time()
 
 return(f'"Il numero di triangoli nel grafo è:" {numeroTriangoli})'
        f'"Per calcolare il numero di triangoli nel grafo sono stati necessari minuti:" {round((end-start)/60,2)})')
 

In [18]:
calcola_triangoli(fromtoID)

'"Il numero di triangoli nel grafo è:" 494728)"Per calcolare il numero di triangoli nel grafo sono stati necessari minuti:" 6.13)'